In [ ]:
def main_learning(code, best_C, best_gamma, test_size=0.05, k=1):
    from sklearn.model_selection import train_test_split
    from sklearn.svm import SVC
    from sklearn.metrics import confusion_matrix
    import time
    from sklearn.model_selection import KFold
    
    _C =1.0*10**best_C
    _gamma = 1.0*10**best_gamma    
    accuracy=0
    cnf_matrix=np.zeros((3,3))

    if(k>1):
        kf = KFold(n_splits = k, shuffle = False)
        for train_index, test_index in kf.split(y):
            X_train, X_test, y_train, y_test =X[train_index], X[test_index], y[train_index], y[test_index]
            classifier=SVC(kernel='rbf',C=_C, gamma=_gamma)
            start=time.time()
            fit=classifier.fit(X_train,y_train)
            end=time.time()
            time_score=end-start
            _accuracy=classifier.score(X_test, y_test)
            predict_vec = classifier.predict(X_test)
            _cnf_matrix=confusion_matrix(y_test,predict_vec)
            accuracy=accuracy+_accuracy
            cnf_matrix+=np.array(_cnf_matrix)
            
            with open('classifier_'+code+'.pickle','wb') as f:
                pickle.dump((classifier,z[train_index],z[test_index]),f,protocol=4)
    elif(k==1):
        X_train, X_test, y_train, y_test, z_train, z_test = train_test_split(X,y,z,test_size=test_size)
        classifier=SVC(kernel='rbf',C=_C, gamma=_gamma)
        start=time.time()
        fit=classifier.fit(X_train,y_train)
        end=time.time()
        time_score=end-start
        accuracy=classifier.score(X_test, y_test)
        predict_vec = classifier.predict(X_test)
        cnf_matrix=confusion_matrix(y_test,predict_vec)
        with open('classifier_'+code+'.pickle','wb') as f:
            pickle.dump((classifier,z_train,z_test),f,protocol=4)
    accuracy=accuracy/k
    cnf_matrix=cnf_matrix/(k*len(y_test))
    return(accuracy, time_score, cnf_matrix)


class_names=['Null','Dead','Live']
import itertools
def plot_confusion_matrix(cm, classes,title='Confusion matrix',cmap=plt.cm.Blues):
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' 
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black", 
                 fontsize=15)

    plt.tight_layout()
    plt.ylabel('True label',fontsize=20)
    plt.xlabel('Predicted label',fontsize=20)

plt.savefig("confusion_matrix.png",dpi=300)


In [ ]:
with open('score_params.pickle','rb') as f:
    lists = pickle.load(f)
[print(_list) for _list in lists];

In [ ]:
images_list=['FUNIST16','FUNIST32']
func_list=['feature_img', 'feature_normal_img', 'feature_canny','feature_lbp_raw', 'feature_lbp', 'feature_hog','feature_hog_raw']
k=0

for _list in lists:
        images=lists[k]["images"]
        func=lists[k]["func"]
        best_C=lists[k]["best_C"]
        best_gamma=lists[k]["best_gamma"]
        print(images, func, best_C, best_gamma)
        with open('pickle\\'+func+images+'.pickle','rb') as f:
            X,y,z=pickle.load(f)
        loop=0
        while(loop<3):
            _accuracy, _time, _cnf_matrix = true_learning(code=str(k), best_C=best_C, best_gamma=best_gamma, test_size=0.05, k=1)
            loop+=1
            print(accuracy, time)
            cnf_matrix+=_cnf_matrix
            accuracy+=_accuracy
            time+=_time
            
        cnf_matrix=cnf_matrix/loop
        accuracy=accuracy/loop
        time=time/loop
        
        plt.figure()
        sns.heatmap(cnf_matrix)
        plt.show()
        
        k=k+1